#### Imports

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from Project import Project

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from itertools import product
import pickle
import pandas as pd
import numpy as np

from Scripts.neuralnets.NNPreinstalledModelSelection import *
from Scripts.neuralnets.NNTrainingUtils import train, kfold_train
from Scripts.neuralnets.NNTrainingUtils import train_autoencoder
from Scripts.datahandling.BenchmarkDataset import BenchmarkDataset
from Scripts.datahandling.Dataloader import Dataloader
from Models.Autoencoder1 import Autoencoder

ngpu = 1; device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [19]:
batch_size = 32
signal_length = 1000
dataset = BenchmarkDataset(Project, signal_length, classes = list(range(1, 5 + 1)))
dataloaders, dataset_sizes = Dataloader(dataset, batch_size)
print(dataset_sizes)

{'train': 1632, 'val': 408}


***

#### Training Directly With a CNN

In [11]:
def model_initializer():
    model_type = 'resnet'; model_size = '18'; num_classes = 40; model_name = model_type + model_size
    model = model_selection(model_name, num_classes, pretrained = True)
    model.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
    model = model.to(device)
    return model

def optimizer_initializer(model):
    return optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

def scheduler_initializer(optimizer):
    return optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

In [12]:
# optimizer = optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9)
criterion = nn.CrossEntropyLoss()

In [9]:
sample = next(iter(dataloaders['train']))

In [10]:
sample[1]

tensor([ 6, 34, 10, 26, 15, 28,  2, 21,  8, 32,  9, 14, 39, 27,  2, 23, 17, 31,
        15, 38, 24, 32, 12, 18, 17, 14, 33,  3, 36,  9, 27, 22])

In [13]:
name = 'Resnet18 - Pretrained - {} Classes - Adam - {} - Individual {}'.format(5, 500, 1)

model, stats = kfold_train(Project, model_initializer, dataset, criterion, optimizer_initializer, num_epochs = 50, num_folds = 5, batch_size = batch_size, log_name = name)

***

#### Training With Autoencoders

Parameters

stride - s: \[1, 2, 4, 8, 16\]  
kernel - k: \[3, 5, 7, 9, 11\]  
feature_maps - fm: \[1, 3\]  

In [31]:
model = autoencoder_initializer(1, 4, 0., 1)
summary(model, (1, 64, int(signal_length)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Dropout2d-1          [-1, 1, 64, 1000]               0
            Conv2d-2           [-1, 1, 64, 250]               4
              ReLU-3           [-1, 1, 64, 250]               0
         MaxPool2d-4  [[-1, 1, 64, 125], [-1, 1, 64, 125]]               0
       BatchNorm2d-5           [-1, 1, 64, 125]               2
       MaxUnpool2d-6           [-1, 1, 64, 250]               0
   ConvTranspose2d-7          [-1, 1, 64, 1000]               4
Total params: 10
Trainable params: 10
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.24
Forward/backward pass size (MB): 486.88
Params size (MB): 0.00
Estimated Total Size (MB): 487.12
----------------------------------------------------------------


In [9]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9)
# optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

In [ ]:
name = ""

In [14]:
model, stats = train_autoencoder(Project, model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs = 100)

***

In [16]:
X = next(iter(dataloaders['train']))[0]

In [20]:
class HybridModel(nn.Module):
    def __init__(self, classifier, encoder):
        super(HybridModel, self).__init__()
        self.encoder = encoder
        self.classifier = classifier
        self.encoder.eval()
#     def normalize(self, X):
#         mean = torch.mean(X, dim = 3).unsqueeze(3).expand_as(X)
#         std  = torch.std(X, dim = 3).unsqueeze(3).expand_as(X)
#         return (X - mean)/std
        
    def forward(self, x):
        with torch.set_grad_enabled(False):
#             x = self.normalize(self.encoder(x))
            x = self.encoder(x)
            
        x = self.classifier(x)
        
        return x

In [21]:
model_type = 'resnet'; model_size = '18'; num_classes = 5; model_name = model_type + model_size
classifier = model_selection(model_name, num_classes)
classifier.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
encoder = model.encoder
hybrid = HybridModel(classifier, encoder).to(device)

In [17]:
# optimizer = optim.SGD(hybrid.parameters(), lr = 1e-3, momentum = 0.9)
optimizer = optim. Adam(hy)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
criterion = nn.CrossEntropyLoss()

In [ ]:
hybrid, stats = kfold_train(Project, hybrid, dataset, criterion, optimizer, num_epochs = 10, num_folds = 5)

***

In [21]:
class Autoencoder(nn.Module):
    def __init__(self, stride, kernel_n, dropout_p, n_feature_maps = 3):
        super(Autoencoder, self).__init__()
        n1 = kernel_n
        s = stride
        self.encoder = nn.Sequential(
            nn.Dropout2d(dropout_p),
            nn.Conv2d(1, n_feature_maps, kernel_size = (1, 2*n1 + 1), padding = (0, n1), stride = (1, s)),
            nn.ReLU(inplace = True),
        )
        
        self.pool    = nn.MaxPool2d(kernel_size = (1, 2), stride = (1, 2), return_indices = True)
        self.batch_norm = nn.BatchNorm2d(n_feature_maps)
        self.unpool  = nn.MaxUnpool2d(kernel_size = (1, 2), stride = (1, 2))
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(n_feature_maps, 1, kernel_size = (1, 2*n1 + 1), padding = (0, n1), stride = (1, s), output_padding = (0, s - 1))
        )
        
    
    def forward(self, x):
        x = self.encoder(x)
        x, _2 = self.pool(x)
        x = self.batch_norm(x)
        x = self.unpool(x, _2)
        x = self.decoder(x)
        
        return x
    
    def encode(self, x):
        with torch.set_grad_enabled(False):
            x = self.encoder(x)
            x, _ = self.pool(x)
            x = self.batch_norm(x)
            
        return x


In [29]:
def autoencoder_initializer(kernel_n, stride, dropout_p, n_feature_maps):
    model = Autoencoder(kernel_n = kernel_n, stride = stride, dropout_p = dropout_p, n_feature_maps = n_feature_maps).to(device)
    return model

In [23]:
def hybrid_initializer():
    classifier = model_selection(model_name, num_classes)
    classifier.conv1 = nn.Conv2d(fm, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
    encoder = model.encoder
    hybrid = HybridModel(classifier, encoder).to(device)
    return hybrid

In [24]:
def optimizer_initializer(model):
    return optim.SGD(model.classifier.parameters(), lr = 1e-3, momentum = 0.9)
#     return optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

def scheduler_initializer(optimizer):
    return optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)

In [32]:
S = [2, 4]
K = [1]
FM = [1, 3]
DP = [0]
model_type = 'resnet'; model_size = '18'; num_classes = 5; model_name = model_type + model_size

for (s, k, fm, dp) in product(S, K, FM, DP):
    # Autoencoder Training
    model = Autoencoder(kernel_n = k, stride = s, dropout_p = dp, n_feature_maps = fm).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)
    name = '1000 - Autoencoder - s = {} k = {} fm = {} dp = {} - 5 classes'.format(s, k, fm, dp)
    model, stats = train_autoencoder(Project, model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs = 50, log_name = name, scheduler = scheduler)
    
    with open('Outputs/'+name+' - stats', 'wb') as stats_file:
        pickle.dump(stats, stats_file)
        
    # Classifier training

    criterion = nn.CrossEntropyLoss()
    name = 'ResNet50 - TL - ' + name
    hybrid, stats = kfold_train(Project, hybrid_initializer, dataset, criterion, optimizer_initializer, num_epochs = 50, num_folds = 5, log_name = name)
    
    with open('Outputs/'+name+' - stats', 'wb') as stats_file:
        pickle.dump(stats, stats_file)
    
    del model
    del hybrid
    del stats